# FEATURE ENGINEERING ELECTIONS DATA

Importing nessecary libraries and reading cleaned data

In [1]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd

In [2]:
presid=pd.read_csv('data/cleaned_1976-2020-president.csv')
presid.drop(columns=['state_cen','state_ic'],inplace=True)
presid.rename(columns={'state_fips':'fips'},inplace=True)

In [3]:
presid.head()

,year,state,state_po,fips,candidate,candidatevotes,totalvotes,party
0,1976,ALABAMA,AL,1,"CARTER, JIMMY",659170,1182850,DEMOCRAT
1,1976,ALABAMA,AL,1,"FORD, GERALD",504070,1182850,REPUBLICAN
2,1976,ALASKA,AK,2,"FORD, GERALD",71555,123574,REPUBLICAN
3,1976,ALASKA,AK,2,"CARTER, JIMMY",44058,123574,DEMOCRAT
4,1976,ARIZONA,AZ,4,"FORD, GERALD",418642,742719,REPUBLICAN


In [4]:
presid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            1224 non-null   int64 
 1   state           1224 non-null   object
 2   state_po        1224 non-null   object
 3   fips            1224 non-null   int64 
 4   candidate       1224 non-null   object
 5   candidatevotes  1224 non-null   int64 
 6   totalvotes      1224 non-null   int64 
 7   party           1224 non-null   object
dtypes: int64(4), object(4)
memory usage: 76.6+ KB


Let's store the names of the candidates here in case they are needed later, and delete them from the dataset

In [5]:
candidates=presid.groupby(['year','party'])['candidate'].first()
candidates

year  party     
1976  DEMOCRAT            CARTER, JIMMY
      REPUBLICAN           FORD, GERALD
1980  DEMOCRAT            CARTER, JIMMY
      REPUBLICAN         REAGAN, RONALD
1984  DEMOCRAT          MONDALE, WALTER
      REPUBLICAN         REAGAN, RONALD
1988  DEMOCRAT         DUKAKIS, MICHAEL
      REPUBLICAN      BUSH, GEORGE H.W.
1992  DEMOCRAT            CLINTON, BILL
      REPUBLICAN      BUSH, GEORGE H.W.
1996  DEMOCRAT            CLINTON, BILL
      REPUBLICAN           DOLE, ROBERT
2000  DEMOCRAT                 GORE, AL
      REPUBLICAN        BUSH, GEORGE W.
2004  DEMOCRAT              KERRY, JOHN
      REPUBLICAN        BUSH, GEORGE W.
2008  DEMOCRAT         OBAMA, BARACK H.
      REPUBLICAN           MCCAIN, JOHN
2012  DEMOCRAT         OBAMA, BARACK H.
      REPUBLICAN           ROMNEY, MITT
2016  DEMOCRAT         CLINTON, HILLARY
      REPUBLICAN       TRUMP, DONALD J.
2020  DEMOCRAT      BIDEN, JOSEPH R. JR
      REPUBLICAN       TRUMP, DONALD J.
Name: candidate, dtype:

In [6]:
presid.drop(columns=['candidate'],inplace=True)

Transforming the DataFrame to leave one row for each year-state combination and put Democratic and Republican votes in separate columns. This is achieved by splitting the dataset into two parts, renaming the 'candidatevotes' column, dropping 'party' column and merging them back together.

In [7]:
dem_df=presid[presid.party=='DEMOCRAT'].rename(columns={'candidatevotes':'d_votes'}).drop(columns=['party'])
rep_df=presid[presid.party=='REPUBLICAN'].rename(columns={'candidatevotes':'r_votes'}).drop(columns=['party'])
presid=pd.merge(dem_df, rep_df, on = ['year','state','state_po','fips','totalvotes'], how = "outer")

In [8]:
presid.head()

,year,state,state_po,fips,d_votes,totalvotes,r_votes
0,1976,ALABAMA,AL,1,659170,1182850,504070
1,1976,ALASKA,AK,2,44058,123574,71555
2,1976,ARIZONA,AZ,4,295602,742719,418642
3,1976,ARKANSAS,AR,5,498604,767535,267903
4,1976,CALIFORNIA,CA,6,3742284,7803770,3882244


In [9]:
presid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        612 non-null    int64 
 1   state       612 non-null    object
 2   state_po    612 non-null    object
 3   fips        612 non-null    int64 
 4   d_votes     612 non-null    int64 
 5   totalvotes  612 non-null    int64 
 6   r_votes     612 non-null    int64 
dtypes: int64(5), object(2)
memory usage: 33.6+ KB


Now, since numbers of votes are different state-by-state and in time, it's better to analyse them comparatively. Here, new values are created by calculating proportions of votes and differences between them for each year and state. By convention, Democratic votes will go first, so if the value of 'dr_percent' is positive, then the Democratic candidate won the election in that exact state and year, otherwise, the Republican candidate won.

In [10]:
presid['d_percent']=(presid.d_votes/presid.totalvotes)*100
presid['r_percent']=(presid.r_votes/presid.totalvotes)*100
presid['dr_percent']=presid['d_percent']-presid['r_percent']

In [11]:
presid.head()

,year,state,state_po,fips,d_votes,totalvotes,r_votes,d_percent,r_percent,dr_percent
0,1976,ALABAMA,AL,1,659170,1182850,504070,55.727269,42.614871,13.112398
1,1976,ALASKA,AK,2,44058,123574,71555,35.653131,57.904575,-22.251444
2,1976,ARIZONA,AZ,4,295602,742719,418642,39.799978,56.366136,-16.566158
3,1976,ARKANSAS,AR,5,498604,767535,267903,64.961728,34.904337,30.057392
4,1976,CALIFORNIA,CA,6,3742284,7803770,3882244,47.954822,49.748314,-1.793492


Adding a new column that will show the value of 'dr_percent' in the same state in the previous election. Obviously, these values for 1976 are going to be meaningless, since we don't have the data for 1972 here.

In [12]:
presid['prev_dr_percent']=np.append(100*np.ones(shape=(51,)), presid.dr_percent.to_numpy()[:-51])

In [13]:
presid.tail()

,year,state,state_po,fips,d_votes,totalvotes,r_votes,d_percent,r_percent,dr_percent,prev_dr_percent
607,2020,VIRGINIA,VA,51,2413568,4460524,1962430,54.109517,43.995504,10.114014,5.323706
608,2020,WASHINGTON,WA,53,2369612,4087631,1584651,57.970301,38.766978,19.203323,15.706000
609,2020,WEST VIRGINIA,WV,54,235984,794652,545382,29.696521,68.631552,-38.935031,-42.153647
610,2020,WISCONSIN,WI,55,1630866,3298041,1610184,49.449537,48.822437,0.627100,-0.764343
611,2020,WYOMING,WY,56,73491,278503,193559,26.387867,69.499790,-43.111923,-45.769510


In [14]:
presid[(presid.year!=1976) & ((presid.dr_percent*presid.prev_dr_percent)<0)].sort_values(by='prev_dr_percent')

,year,state,state_po,fips,d_votes,totalvotes,r_votes,d_percent,r_percent,dr_percent,prev_dr_percent
233,1992,NEW HAMPSHIRE,NH,33,209040,537215,202484,38.911795,37.691427,1.220368,-26.156373
232,1992,NEVADA,NV,32,189148,506318,175828,37.357550,34.726792,2.630758,-21.362562
422,2008,INDIANA,IN,18,1374039,2751054,1345648,49.945912,48.913907,1.032004,-20.681790
214,1992,GEORGIA,GA,13,1008966,2321133,995252,43.468685,42.877853,0.590832,-20.254444
246,1992,TENNESSEE,TN,47,933521,1982638,841300,47.084793,42.433364,4.651429,-16.344629
...,...,...,...,...,...,...,...,...,...,...,...
354,2000,WEST VIRGINIA,WV,54,295497,648124,336475,45.592664,51.915220,-6.322556,14.748161
112,1984,GEORGIA,GA,13,706628,1776103,1068722,39.785305,60.172299,-20.386993,14.828799
72,1980,MASSACHUSETTS,MA,25,1053802,2524090,1057631,41.749779,41.901477,-0.151698,15.669869
309,2000,ARKANSAS,AR,5,422768,921781,472940,45.864256,51.307198,-5.442941,16.935591


Checking the correctness of the 'prev_dr_percent' value in every row from year 1980 onwards.

In [15]:
good=True
for year in range(1980,2021,4):
    for state in presid.state.unique():
        if ((presid[(presid.year==year) & (presid.state==state)].prev_dr_percent.iloc[0]) 
           != (presid[(presid.year==year-4) & (presid.state==state)].dr_percent.iloc[0])):
            good=False
            break

print(good)

True


The compared values have been equal every time, therefore, they are correct.

Adding the parameter showing the swing: how have the results in the state changed since the previous election. Again, if this parameter is positive, then the difference between the vote percentages has improved for Democrats, otherwise it has improved for Republicans.

In [16]:
presid['dr_swing']=presid.dr_percent-presid.prev_dr_percent

In [17]:
presid.tail()

,year,state,state_po,fips,d_votes,totalvotes,r_votes,d_percent,r_percent,dr_percent,prev_dr_percent,dr_swing
607,2020,VIRGINIA,VA,51,2413568,4460524,1962430,54.109517,43.995504,10.114014,5.323706,4.790308
608,2020,WASHINGTON,WA,53,2369612,4087631,1584651,57.970301,38.766978,19.203323,15.706000,3.497324
609,2020,WEST VIRGINIA,WV,54,235984,794652,545382,29.696521,68.631552,-38.935031,-42.153647,3.218616
610,2020,WISCONSIN,WI,55,1630866,3298041,1610184,49.449537,48.822437,0.627100,-0.764343,1.391443
611,2020,WYOMING,WY,56,73491,278503,193559,26.387867,69.499790,-43.111923,-45.769510,2.657587


Calculating the national vote totals for every election in a new DataFrame. 

In [18]:
presid_national=pd.DataFrame(presid.groupby('year')[['d_votes','r_votes','totalvotes']].sum()).reset_index()
presid_national

,year,d_votes,r_votes,totalvotes
0,1976,40825839,39145771,81601344
1,1980,35480948,43898770,86496851
2,1984,37568137,54455073,92654861
3,1988,41809074,48886097,91586825
4,1992,44954303,39103872,104599780
5,1996,47401898,39198482,96389818
6,2000,50996062,50456169,105593982
7,2004,59028079,62028285,122349450
8,2008,69498459,59948283,131419253
9,2012,65899660,60932152,129139997


To note, the national vote totals calculated here do not match the final official results from 2020, which is presumably due to inacuracies in the MIT dataset. For example, the official number of total votes cast is 158,429,631. However, for the purposes of this project, this difference of 0.1% is negligible.


Adding the same relative features as in the original dataset. 

In [19]:
presid_national['d_percent']=(presid_national.d_votes/presid_national.totalvotes)*100
presid_national['r_percent']=(presid_national.r_votes/presid_national.totalvotes)*100
presid_national['dr_percent']=presid_national['d_percent']-presid_national['r_percent']

In [20]:
presid_national

,year,d_votes,r_votes,totalvotes,d_percent,r_percent,dr_percent
0,1976,40825839,39145771,81601344,50.030841,47.971968,2.058873
1,1980,35480948,43898770,86496851,41.019930,50.751871,-9.731940
2,1984,37568137,54455073,92654861,40.546321,58.771955,-18.225634
3,1988,41809074,48886097,91586825,45.649660,53.376779,-7.727119
4,1992,44954303,39103872,104599780,42.977436,37.384277,5.593158
5,1996,47401898,39198482,96389818,49.177288,40.666621,8.510667
6,2000,50996062,50456169,105593982,48.294478,47.783186,0.511291
7,2004,59028079,62028285,122349450,48.245480,50.697641,-2.452161
8,2008,69498459,59948283,131419253,52.883012,45.616058,7.266953
9,2012,65899660,60932152,129139997,51.029628,47.183021,3.846607


In [21]:
presid_national['prev_dr_percent']=np.append(100, presid_national.dr_percent.to_numpy()[:-1])
presid_national['dr_swing']=presid_national.dr_percent-presid_national.prev_dr_percent

In [22]:
presid_national

,year,d_votes,r_votes,totalvotes,d_percent,r_percent,dr_percent,prev_dr_percent,dr_swing
0,1976,40825839,39145771,81601344,50.030841,47.971968,2.058873,100.000000,-97.941127
1,1980,35480948,43898770,86496851,41.019930,50.751871,-9.731940,2.058873,-11.790813
2,1984,37568137,54455073,92654861,40.546321,58.771955,-18.225634,-9.731940,-8.493694
3,1988,41809074,48886097,91586825,45.649660,53.376779,-7.727119,-18.225634,10.498515
4,1992,44954303,39103872,104599780,42.977436,37.384277,5.593158,-7.727119,13.320277
5,1996,47401898,39198482,96389818,49.177288,40.666621,8.510667,5.593158,2.917509
6,2000,50996062,50456169,105593982,48.294478,47.783186,0.511291,8.510667,-7.999375
7,2004,59028079,62028285,122349450,48.245480,50.697641,-2.452161,0.511291,-2.963453
8,2008,69498459,59948283,131419253,52.883012,45.616058,7.266953,-2.452161,9.719115
9,2012,65899660,60932152,129139997,51.029628,47.183021,3.846607,7.266953,-3.420347


Dropping the raw vote totals and the 'prev_dr_percent' column, since they won't be needed anymore, and renaming the columns to explicitly show that this is national data.

In [23]:
presid_national.drop(columns=['d_votes','r_votes','totalvotes','prev_dr_percent'],inplace=True)
presid_national.rename(columns={'d_percent':'d_percent_national',
                        'r_percent':'r_percent_national',
                        'dr_percent':'dr_percent_national',
                        'dr_swing':'dr_swing_national'},inplace=True)

In [24]:
presid_national

,year,d_percent_national,r_percent_national,dr_percent_national,dr_swing_national
0,1976,50.030841,47.971968,2.058873,-97.941127
1,1980,41.019930,50.751871,-9.731940,-11.790813
2,1984,40.546321,58.771955,-18.225634,-8.493694
3,1988,45.649660,53.376779,-7.727119,10.498515
4,1992,42.977436,37.384277,5.593158,13.320277
5,1996,49.177288,40.666621,8.510667,2.917509
6,2000,48.294478,47.783186,0.511291,-7.999375
7,2004,48.245480,50.697641,-2.452161,-2.963453
8,2008,52.883012,45.616058,7.266953,9.719115
9,2012,51.029628,47.183021,3.846607,-3.420347


Adding a few features with boolean values that account for obvious info about any presidential election.

In [25]:
presid_national['d_winner_4y_ago']=pd.Series([0,1,0,0,0,1,1,0,0,1,1,0])
presid_national['r_winner_4y_ago']=np.abs(presid_national['d_winner_4y_ago']-1)
presid_national['d_winner_8y_ago']=pd.Series([0,0,1,0,0,0,1,1,0,0,1,1])
presid_national['r_winner_8y_ago']=np.abs(presid_national['d_winner_8y_ago']-1)
presid_national['incumbent_running']=pd.Series([1,1,1,0,1,1,0,1,0,1,0,1])
presid_national['incumbent_not_running']=np.abs(presid_national['incumbent_running']-1)

In [26]:
presid_national

,year,d_percent_national,r_percent_national,dr_percent_national,dr_swing_national,d_winner_4y_ago,r_winner_4y_ago,d_winner_8y_ago,r_winner_8y_ago,incumbent_running,incumbent_not_running
0,1976,50.030841,47.971968,2.058873,-97.941127,0,1,0,1,1,0
1,1980,41.019930,50.751871,-9.731940,-11.790813,1,0,0,1,1,0
2,1984,40.546321,58.771955,-18.225634,-8.493694,0,1,1,0,1,0
3,1988,45.649660,53.376779,-7.727119,10.498515,0,1,0,1,0,1
4,1992,42.977436,37.384277,5.593158,13.320277,0,1,0,1,1,0
5,1996,49.177288,40.666621,8.510667,2.917509,1,0,0,1,1,0
6,2000,48.294478,47.783186,0.511291,-7.999375,1,0,1,0,0,1
7,2004,48.245480,50.697641,-2.452161,-2.963453,0,1,1,0,1,0
8,2008,52.883012,45.616058,7.266953,9.719115,0,1,0,1,0,1
9,2012,51.029628,47.183021,3.846607,-3.420347,1,0,0,1,1,0


Adding the national values to the original dataset

In [27]:
presid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             612 non-null    int64  
 1   state            612 non-null    object 
 2   state_po         612 non-null    object 
 3   fips             612 non-null    int64  
 4   d_votes          612 non-null    int64  
 5   totalvotes       612 non-null    int64  
 6   r_votes          612 non-null    int64  
 7   d_percent        612 non-null    float64
 8   r_percent        612 non-null    float64
 9   dr_percent       612 non-null    float64
 10  prev_dr_percent  612 non-null    float64
 11  dr_swing         612 non-null    float64
dtypes: float64(5), int64(5), object(2)
memory usage: 57.5+ KB


In [28]:
presid_final=pd.merge(presid, presid_national, on = ['year'], how = "outer")

In [29]:
presid_final

,year,state,state_po,fips,d_votes,totalvotes,r_votes,d_percent,r_percent,dr_percent,...,d_percent_national,r_percent_national,dr_percent_national,dr_swing_national,d_winner_4y_ago,r_winner_4y_ago,d_winner_8y_ago,r_winner_8y_ago,incumbent_running,incumbent_not_running
0,1976,ALABAMA,AL,1,659170,1182850,504070,55.727269,42.614871,13.112398,...,50.030841,47.971968,2.058873,-97.941127,0,1,0,1,1,0
1,1976,ALASKA,AK,2,44058,123574,71555,35.653131,57.904575,-22.251444,...,50.030841,47.971968,2.058873,-97.941127,0,1,0,1,1,0
2,1976,ARIZONA,AZ,4,295602,742719,418642,39.799978,56.366136,-16.566158,...,50.030841,47.971968,2.058873,-97.941127,0,1,0,1,1,0
3,1976,ARKANSAS,AR,5,498604,767535,267903,64.961728,34.904337,30.057392,...,50.030841,47.971968,2.058873,-97.941127,0,1,0,1,1,0
4,1976,CALIFORNIA,CA,6,3742284,7803770,3882244,47.954822,49.748314,-1.793492,...,50.030841,47.971968,2.058873,-97.941127,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,2020,VIRGINIA,VA,51,2413568,4460524,1962430,54.109517,43.995504,10.114014,...,51.264540,46.815648,4.448892,2.351825,0,1,1,0,1,0
608,2020,WASHINGTON,WA,53,2369612,4087631,1584651,57.970301,38.766978,19.203323,...,51.264540,46.815648,4.448892,2.351825,0,1,1,0,1,0
609,2020,WEST VIRGINIA,WV,54,235984,794652,545382,29.696521,68.631552,-38.935031,...,51.264540,46.815648,4.448892,2.351825,0,1,1,0,1,0
610,2020,WISCONSIN,WI,55,1630866,3298041,1610184,49.449537,48.822437,0.627100,...,51.264540,46.815648,4.448892,2.351825,0,1,1,0,1,0


In [30]:
presid_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   612 non-null    int64  
 1   state                  612 non-null    object 
 2   state_po               612 non-null    object 
 3   fips                   612 non-null    int64  
 4   d_votes                612 non-null    int64  
 5   totalvotes             612 non-null    int64  
 6   r_votes                612 non-null    int64  
 7   d_percent              612 non-null    float64
 8   r_percent              612 non-null    float64
 9   dr_percent             612 non-null    float64
 10  prev_dr_percent        612 non-null    float64
 11  dr_swing               612 non-null    float64
 12  d_percent_national     612 non-null    float64
 13  r_percent_national     612 non-null    float64
 14  dr_percent_national    612 non-null    float64
 15  dr_swi

Adding new features that will highlight the party strength in a given state relative to the national results that year.

In [31]:
presid_final['d_percent_vs_national']=presid_final.d_percent-presid_final.d_percent_national
presid_final['r_percent_vs_national']=presid_final.r_percent-presid_final.r_percent_national
presid_final['dr_percent_vs_national']=presid_final.dr_percent-presid_final.dr_percent_national
presid_final['dr_swing_vs_national']=presid_final.dr_swing-presid_final.dr_swing_national

In [32]:
presid_final.drop(columns=['d_votes','r_votes','totalvotes','prev_dr_percent'],inplace=True)
presid_final.drop(columns=['d_percent_national','r_percent_national','dr_percent_national','dr_swing_national'],inplace=True)

Dropping the data for 1976, since the swing features require knowledge of previous election results.

In [33]:
presid_final=presid_final[presid_final.year!=1976]

In [34]:
presid_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 561 entries, 51 to 611
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    561 non-null    int64  
 1   state                   561 non-null    object 
 2   state_po                561 non-null    object 
 3   fips                    561 non-null    int64  
 4   d_percent               561 non-null    float64
 5   r_percent               561 non-null    float64
 6   dr_percent              561 non-null    float64
 7   dr_swing                561 non-null    float64
 8   d_winner_4y_ago         561 non-null    int64  
 9   r_winner_4y_ago         561 non-null    int64  
 10  d_winner_8y_ago         561 non-null    int64  
 11  r_winner_8y_ago         561 non-null    int64  
 12  incumbent_running       561 non-null    int64  
 13  incumbent_not_running   561 non-null    int64  
 14  d_percent_vs_national   561 non-null    float6

In [35]:
presid_final.tail()

,year,state,state_po,fips,d_percent,r_percent,dr_percent,dr_swing,d_winner_4y_ago,r_winner_4y_ago,d_winner_8y_ago,r_winner_8y_ago,incumbent_running,incumbent_not_running,d_percent_vs_national,r_percent_vs_national,dr_percent_vs_national,dr_swing_vs_national
607,2020,VIRGINIA,VA,51,54.109517,43.995504,10.114014,4.790308,0,1,1,0,1,0,2.844977,-2.820144,5.665121,2.438483
608,2020,WASHINGTON,WA,53,57.970301,38.766978,19.203323,3.497324,0,1,1,0,1,0,6.705761,-8.048670,14.754431,1.145498
609,2020,WEST VIRGINIA,WV,54,29.696521,68.631552,-38.935031,3.218616,0,1,1,0,1,0,-21.568019,21.815904,-43.383923,0.866791
610,2020,WISCONSIN,WI,55,49.449537,48.822437,0.627100,1.391443,0,1,1,0,1,0,-1.815003,2.006789,-3.821793,-0.960382
611,2020,WYOMING,WY,56,26.387867,69.499790,-43.111923,2.657587,0,1,1,0,1,0,-24.876674,22.684142,-47.560815,0.305762


# FEATURE ENGINEERING ECONOMIC DATA 

Reading the sainc file, containing all the economic data from the sainc30, sainc35, sainc4 and sainc50 notebooks.

In [36]:
sainc=pd.read_csv('data/sainc.csv')

In [37]:
sainc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5411 entries, 0 to 5410
Data columns (total 50 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fips       5411 non-null   object 
 1   state      5411 non-null   object 
 2   TableName  5411 non-null   object 
 3   code       5411 non-null   object 
 4   features   5411 non-null   object 
 5   1976       5411 non-null   float64
 6   1977       5411 non-null   float64
 7   1978       5411 non-null   float64
 8   1979       5411 non-null   float64
 9   1980       5411 non-null   float64
 10  1981       5411 non-null   float64
 11  1982       5411 non-null   float64
 12  1983       5411 non-null   float64
 13  1984       5411 non-null   float64
 14  1985       5411 non-null   float64
 15  1986       5411 non-null   float64
 16  1987       5411 non-null   float64
 17  1988       5411 non-null   float64
 18  1989       5411 non-null   float64
 19  1990       5411 non-null   float64
 20  1991    

Only the data for the election years and the years right before an election is needed. Dropping the data for the other years.

In [38]:
cols_to_drop=np.union1d(np.arange(1977,2021,4),np.arange(1978,2021,4))
cols_to_drop

array([1977, 1978, 1981, 1982, 1985, 1986, 1989, 1990, 1993, 1994, 1997,
       1998, 2001, 2002, 2005, 2006, 2009, 2010, 2013, 2014, 2017, 2018])

In [39]:
sainc.drop(columns=[str(x) for x in cols_to_drop],inplace=True)

In [40]:
sainc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5411 entries, 0 to 5410
Data columns (total 28 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fips       5411 non-null   object 
 1   state      5411 non-null   object 
 2   TableName  5411 non-null   object 
 3   code       5411 non-null   object 
 4   features   5411 non-null   object 
 5   1976       5411 non-null   float64
 6   1979       5411 non-null   float64
 7   1980       5411 non-null   float64
 8   1983       5411 non-null   float64
 9   1984       5411 non-null   float64
 10  1987       5411 non-null   float64
 11  1988       5411 non-null   float64
 12  1991       5411 non-null   float64
 13  1992       5411 non-null   float64
 14  1995       5411 non-null   float64
 15  1996       5411 non-null   float64
 16  1999       5411 non-null   float64
 17  2000       5411 non-null   float64
 18  2003       5411 non-null   float64
 19  2004       5411 non-null   float64
 20  2007    

Removing the extra headers, that are in the dataset because it has been created from multiple notebooks, and removing unnecessary stars from state names.

In [41]:
sainc[['fips','state']].drop_duplicates()

,fips,state
0,0,United States
38,1,Alabama
76,2,Alaska
114,4,Arizona
152,5,Arkansas
190,6,California
228,8,Colorado
266,9,Connecticut
304,10,Delaware
342,11,District of Columbia


In [42]:
sainc=sainc[sainc.fips!='fips']
sainc=sainc.astype({'fips': 'int'})

In [43]:
def remove_star(x):
    x=x.strip()
    if (x[-1]=='*'):
        return x[:-2]
    else:
        return x

In [44]:
sainc.state = sainc.state.apply(remove_star)

Removing prefixes from feature names

In [45]:
sainc[['TableName','code','features']].drop_duplicates()

,TableName,code,features
0,SAINC30,10,"Personal income (millions of dollars), Million..."
1,SAINC30,45,"Net earnings by place of residence, Millions o..."
2,SAINC30,50,"Personal current transfer receipts, Millions o..."
3,SAINC30,60,"Income maintenance benefits, Millions of dollars"
4,SAINC30,70,"Unemployment insurance compensation, Millions ..."
...,...,...,...
4845,SAINC50,70,"Federal government, Thousands of dollars"
4846,SAINC50,80,"Income taxes (net of refunds), Thousands of do..."
4847,SAINC50,81,"Income taxes (gross), Thousands of dollars"
4848,SAINC50,82,"Less: Refunds, Thousands of dollars"


In [46]:
def remove_prefix(x):
    if (x[:6]=='Equals'):
        return x[8:]
    if ((x[:4]=='Less') or (x[:4]=='Plus')):
        return x[6:]
    return x

In [47]:
sainc.features = sainc.features.apply(remove_prefix)

Showing all feature names and what initial dataset are they from. There are a few duplicates.

In [48]:
grouped=sainc.groupby('features').TableName.unique()
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(grouped)

features
Average earnings per job (dollars), Dollars                                                              [SAINC30]
Average nonfarm proprietors' income, Dollars                                                             [SAINC30]
Average wages and salaries, Dollars                                                                      [SAINC30]
Contributions for government social insurance, Millions of dollars                                        [SAINC4]
Current transfer receipts of individuals from businesses , Thousands of dollars                          [SAINC35]
Current transfer receipts of individuals from governments, Thousands of dollars                          [SAINC35]
Current transfer receipts of nonprofit institutions, Thousands of dollars                                [SAINC35]
Disposable personal income, Thousands of dollars                                                         [SAINC50]
Dividends, interest, and rent, Millions of dollars                     

In [49]:
sainc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5408 entries, 0 to 5410
Data columns (total 28 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fips       5408 non-null   int32  
 1   state      5408 non-null   object 
 2   TableName  5408 non-null   object 
 3   code       5408 non-null   object 
 4   features   5408 non-null   object 
 5   1976       5408 non-null   float64
 6   1979       5408 non-null   float64
 7   1980       5408 non-null   float64
 8   1983       5408 non-null   float64
 9   1984       5408 non-null   float64
 10  1987       5408 non-null   float64
 11  1988       5408 non-null   float64
 12  1991       5408 non-null   float64
 13  1992       5408 non-null   float64
 14  1995       5408 non-null   float64
 15  1996       5408 non-null   float64
 16  1999       5408 non-null   float64
 17  2000       5408 non-null   float64
 18  2003       5408 non-null   float64
 19  2004       5408 non-null   float64
 20  2007       54

Deleting the exact duplications originating from the sainc4 dataset 

In [50]:
sainc=sainc[(sainc.TableName!='SAINC4') | 
            (sainc.features.isin(['Contributions for government social insurance, Millions of dollars',
                                  'Employee and self-employed contributions for government social insurance, Millions of dollars',
                                  'Nonfarm personal income, Millions of dollars']))]

Deleting the features that already exist with different units. 

In [51]:
sainc=sainc[~(sainc.features.isin(['Earned Income Tax Credit (EITC) , Thousands of dollars',
                                  'Income maintenance benefits, Thousands of dollars',
                                  'Per capita personal income (dollars), Dollars',
                                  'Personal current transfer receipts (thousands of dollars), Thousands of dollars',
                                  'Personal income (thousands of dollars), Thousands of dollars',
                                  'Unemployment insurance compensation, Thousands of dollars']))]

Deleting the features that are already calculated per capita.

In [52]:
sainc=sainc[~(sainc.features.isin(['Disposable personal income, Thousands of dollars',
                                  'Dividends, interest, and rent, Millions of dollars',
                                  'Income maintenance benefits, Millions of dollars',
                                  'Net earnings by place of residence, Millions of dollars',
                                  'Personal current transfer receipts, Millions of dollars',
                                  'Personal income (millions of dollars), Millions of dollars',
                                  'Rental income of persons, Millions of dollars',
                                  'Retirement and other, Millions of dollars',
                                  'Unemployment insurance compensation, Millions of dollars',
                                  "Nonfarm proprietors' income, Millions of dollars",
                                  'Wages and salaries, Millions of dollars',
                                  'Earnings by place of work, Millions of dollars']))]

In [53]:
sainc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3640 entries, 7 to 5410
Data columns (total 28 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fips       3640 non-null   int32  
 1   state      3640 non-null   object 
 2   TableName  3640 non-null   object 
 3   code       3640 non-null   object 
 4   features   3640 non-null   object 
 5   1976       3640 non-null   float64
 6   1979       3640 non-null   float64
 7   1980       3640 non-null   float64
 8   1983       3640 non-null   float64
 9   1984       3640 non-null   float64
 10  1987       3640 non-null   float64
 11  1988       3640 non-null   float64
 12  1991       3640 non-null   float64
 13  1992       3640 non-null   float64
 14  1995       3640 non-null   float64
 15  1996       3640 non-null   float64
 16  1999       3640 non-null   float64
 17  2000       3640 non-null   float64
 18  2003       3640 non-null   float64
 19  2004       3640 non-null   float64
 20  2007       36

In [54]:
grouped=sainc.groupby('features').TableName.unique()
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(grouped)

features
Average earnings per job (dollars), Dollars                                                      [SAINC30]
Average nonfarm proprietors' income, Dollars                                                     [SAINC30]
Average wages and salaries, Dollars                                                              [SAINC30]
Contributions for government social insurance, Millions of dollars                                [SAINC4]
Current transfer receipts of individuals from businesses , Thousands of dollars                  [SAINC35]
Current transfer receipts of individuals from governments, Thousands of dollars                  [SAINC35]
Current transfer receipts of nonprofit institutions, Thousands of dollars                        [SAINC35]
Earned Income Tax Credit (EITC), Thousands of dollars                                            [SAINC35]
Education and training assistance , Thousands of dollars                                         [SAINC35]
Employee and self-employed c

In [55]:
sainc.features.unique().size

69

In the end, we have no more duplicates and 69 economic features. This amount was achieved unintentionally.

Dropping the columns showing the original source of data.

In [56]:
sainc.drop(columns=['TableName','code'],inplace=True)

Showing how many rows of the DataFrame correspond to each feature. The correct amount is 52: 50 states + D.C. + national data.

In [57]:
with pd.option_context('display.max_rows', None): 
    print(sainc.features.value_counts())

features
Personal current taxes, Thousands of dollars                                                     104
Personal dividend income, Millions of dollars                                                     52
Excluding family assistance, Thousands of dollars                                                 52
Veterans' pension and disability benefits, Thousands of dollars                                   52
Veterans' benefits, Thousands of dollars                                                          52
Unemployment compensation for veterans (UCX), Thousands of dollars                                52
Unemployment compensation for Federal civilian employees (UCFE), Thousands of dollars             52
Excluding state unemployment insurance compensation, Thousands of dollars                         52
State unemployment insurance compensation, Thousands of dollars                                   52
Family assistance, Thousands of dollars                                           

In [58]:
sainc.drop_duplicates(inplace=True)

After dropping the duplicate rows, the amount of rows is correct, since 52*69=3588

In [59]:
sainc[sainc.features=='Personal current taxes, Thousands of dollars'].shape[0]

52

In [60]:
sainc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3588 entries, 7 to 5410
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fips      3588 non-null   int32  
 1   state     3588 non-null   object 
 2   features  3588 non-null   object 
 3   1976      3588 non-null   float64
 4   1979      3588 non-null   float64
 5   1980      3588 non-null   float64
 6   1983      3588 non-null   float64
 7   1984      3588 non-null   float64
 8   1987      3588 non-null   float64
 9   1988      3588 non-null   float64
 10  1991      3588 non-null   float64
 11  1992      3588 non-null   float64
 12  1995      3588 non-null   float64
 13  1996      3588 non-null   float64
 14  1999      3588 non-null   float64
 15  2000      3588 non-null   float64
 16  2003      3588 non-null   float64
 17  2004      3588 non-null   float64
 18  2007      3588 non-null   float64
 19  2008      3588 non-null   float64
 20  2011      3588 non-null   float64
 

Creating a new DataFrame from all pairs of state and year values. Instead of merging, new features will just be added to this DataFrame as columns after having their values sorted in the same way.

In [61]:
df=sainc[sainc.fips!=0][['fips','state']].drop_duplicates()
years=[str(x) for x in np.arange(1980,2021,4)]
df['year']=np.tile(years,(51,1)).tolist()
df=df.explode(column='year').sort_values(by=['year','fips']).reset_index(drop=True)
df

,fips,state,year
0,1,Alabama,1980
1,2,Alaska,1980
2,4,Arizona,1980
3,5,Arkansas,1980
4,6,California,1980
...,...,...,...
556,51,Virginia,2020
557,53,Washington,2020
558,54,West Virginia,2020
559,55,Wisconsin,2020


The absolute values of the economic features don't matter as much, since most of them are measured in dollars, therefore they are affected by inflation. Moreover, the U.S. states have very different populations, which affects values of economic features that are not caclulated per capita. 

Therefore, out of each economic feature, 4 features for the project were created:
1) 4ych_ : equals the current value (value during the election year) divided by the value during the year of the previous election (4 years previously),
2) 1ych_ : similar to 4ych_, but showing the change from 1 year previously
3) vn_ : equals the current value for a state divided by the current value for the whole U.S.
4) vn_4ych_ : equals the 4ych_ value for a state minus the 4ych_ value for the whole U.S.

All of these values are relative, and show the current value compared to either the national value or the value in the previous years.

In [62]:
all_features=sainc.features.unique()

prev4y_cols=[str(x) for x in np.arange(1976,2017,4)]
prev4y_cols.extend(['fips', 'state'])
prev1y_cols=[str(x) for x in np.arange(1979,2020,4)]
prev1y_cols.extend(['fips', 'state'])
cur_cols=[str(x) for x in np.arange(1980,2021,4)]
cur_cols.extend(['fips', 'state'])

In [63]:
for feat_name in all_features:
    prev4y=sainc[sainc.features==feat_name][prev4y_cols]
    prev1y=sainc[sainc.features==feat_name][prev1y_cols]
    curr=sainc[sainc.features==feat_name][cur_cols]
    
    prev4y=prev4y.melt(id_vars=['fips', 'state']).sort_values(by=['variable','fips'])
    prev1y=prev1y.melt(id_vars=['fips', 'state']).sort_values(by=['variable','fips'])
    curr=curr.melt(id_vars=['fips', 'state']).sort_values(by=['variable','fips'])
    
    curr.rename(columns={'variable':'year','value':'current'},inplace=True)
    curr['1yago']=prev1y.value
    curr['4yago']=prev4y.value
    
    feat=curr[curr.fips!=0]
    feat_national=curr[curr.fips==0].drop(columns=['state','fips'])
    
    feat['4ych_'+feat_name]=feat.current/feat['4yago']
    feat['1ych_'+feat_name]=feat.current/feat['1yago']
    feat.drop(columns=['1yago','4yago'],inplace=True)
    
    feat_national['n_4ych_']=feat_national.current/feat_national['4yago']
    feat_national.rename(columns={'current':'n_current'},inplace=True)
    feat_national.drop(columns=['1yago','4yago'],inplace=True)
    
    feat=pd.merge(feat, feat_national, on = ['year'], how = "outer")
    feat.sort_values(by=['year','fips'],inplace=True)
    
    feat['vn_'+feat_name]=feat.current/feat.n_current
    feat['vn_4ych_'+feat_name]=feat['4ych_'+feat_name]-feat['n_4ych_']
    feat.drop(columns=['current','n_current','n_4ych_'],inplace=True)
    for feat_to_add in feat.columns[3:]:
        df[feat_to_add]=feat[feat_to_add]

In [64]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 279 columns):
 #    Column                                                                                                 Dtype  
---   ------                                                                                                 -----  
 0    fips                                                                                                   int32  
 1    state                                                                                                  object 
 2    year                                                                                                   object 
 3    4ych_Personal dividend income, Millions of dollars                                                     float64
 4    1ych_Personal dividend income, Millions of dollars                                                     float64
 5    vn_Personal dividend income, Millions of dollars                     

Checking for any null or abnornmal values and seeing none

In [65]:
df[(df.isna()).any(axis=1)]

,fips,state,year,"4ych_Personal dividend income, Millions of dollars","1ych_Personal dividend income, Millions of dollars","vn_Personal dividend income, Millions of dollars","vn_4ych_Personal dividend income, Millions of dollars","4ych_Personal interest income, Millions of dollars","1ych_Personal interest income, Millions of dollars","vn_Personal interest income, Millions of dollars",...,"vn_Income taxes (gross), Thousands of dollars","vn_4ych_Income taxes (gross), Thousands of dollars","4ych_Refunds, Thousands of dollars","1ych_Refunds, Thousands of dollars","vn_Refunds, Thousands of dollars","vn_4ych_Refunds, Thousands of dollars","4ych_Local governments, Thousands of dollars","1ych_Local governments, Thousands of dollars","vn_Local governments, Thousands of dollars","vn_4ych_Local governments, Thousands of dollars"


In [66]:
print(f'{df.describe().loc["min"].min():.10f}')

-13.3718147669


In [67]:
df.drop(columns='fips').describe().loc["max"].max()

48.81468613174595

# MERGE AND SAVE DATA

In [68]:
presid_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 561 entries, 51 to 611
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    561 non-null    int64  
 1   state                   561 non-null    object 
 2   state_po                561 non-null    object 
 3   fips                    561 non-null    int64  
 4   d_percent               561 non-null    float64
 5   r_percent               561 non-null    float64
 6   dr_percent              561 non-null    float64
 7   dr_swing                561 non-null    float64
 8   d_winner_4y_ago         561 non-null    int64  
 9   r_winner_4y_ago         561 non-null    int64  
 10  d_winner_8y_ago         561 non-null    int64  
 11  r_winner_8y_ago         561 non-null    int64  
 12  incumbent_running       561 non-null    int64  
 13  incumbent_not_running   561 non-null    int64  
 14  d_percent_vs_national   561 non-null    float6

Dropping some unnecessary columns and changing the type of others to ensure successful merging

In [69]:
presid_final.drop(columns=['state','state_po'],inplace=True)

In [70]:
presid_final=presid_final.astype({'fips': 'int32', 'year' : 'int32'})

In [71]:
df=df.astype({'year' : 'int32'})

Merging economic and election features for every pair of year and state values

In [72]:
df=pd.merge(df, presid_final, on = ['year','fips'], how = "outer")

In [73]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 293 columns):
 #    Column                                                                                                 Dtype  
---   ------                                                                                                 -----  
 0    fips                                                                                                   int32  
 1    state                                                                                                  object 
 2    year                                                                                                   int32  
 3    4ych_Personal dividend income, Millions of dollars                                                     float64
 4    1ych_Personal dividend income, Millions of dollars                                                     float64
 5    vn_Personal dividend income, Millions of dollars                     

Checking if ther are any null values. There should be none

In [74]:
df[(df.isna()).any(axis=1)]

,fips,state,year,"4ych_Personal dividend income, Millions of dollars","1ych_Personal dividend income, Millions of dollars","vn_Personal dividend income, Millions of dollars","vn_4ych_Personal dividend income, Millions of dollars","4ych_Personal interest income, Millions of dollars","1ych_Personal interest income, Millions of dollars","vn_Personal interest income, Millions of dollars",...,d_winner_4y_ago,r_winner_4y_ago,d_winner_8y_ago,r_winner_8y_ago,incumbent_running,incumbent_not_running,d_percent_vs_national,r_percent_vs_national,dr_percent_vs_national,dr_swing_vs_national


Saving the data to be used later

In [75]:
df.to_csv('data/data.csv',index=False)